# File naming web app --> Python/SQL project --> Lasha & Erim

In [1]:
from flask import Flask, render_template, flash, redirect, url_for, session, request, logging
from flask_wtf import FlaskForm
from wtforms import Form, StringField, TextAreaField, PasswordField, validators
from wtforms.validators import DataRequired
from passlib.hash import sha256_crypt
import sqlite3
from flask import g
from functools import wraps
from datetime import datetime
from werkzeug.wrappers import Request, Response

app = Flask(__name__)


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/admin')
def admin():
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute('SELECT history_image.user, history_image.language, COUNT(user.user) FROM user INNER JOIN history_image on user.user = history_image.user group by history_image.user, history_image.language ORDER BY 3 DESC')
    data = c.fetchall()
    c.execute('SELECT history_campaign.user, history_campaign.product, COUNT(user.user) FROM user INNER JOIN history_campaign on user.user = history_campaign.user group by history_campaign.user, history_campaign.product ORDER BY 3 DESC')
    data1 = c.fetchall()
    c.execute('SELECT admin.fullname, history_image.region, COUNT(record_id) FROM admin INNER JOIN history_image on admin.fullname = history_image.user group by admin.fullname, history_image.region order by 3 desc')
    data2 = c.fetchall()
    c.execute('SELECT admin.fullname, history_campaign.region, COUNT(record_id) FROM admin INNER JOIN history_campaign on admin.fullname = history_campaign.user group by admin.fullname, history_campaign.region order by 3 desc')
    data3 = c.fetchall()
    return render_template('admin.html', data=data, data1=data1, data2=data2, data3=data3)


@app.route('/contact')
def contact():
    return render_template('contact.html')

# Defining functions to add generated results to the database
@app.route('/image', methods=['GET', 'POST'])
def image():
    if request.method == 'POST':
        hisi_product = request.form['product']
        hisi_region = request.form['region']
        hisi_image_type = request.form['image_type']
        hisi_language = request.form['language']
        hisi_date = request.form['date']
        hisi_industry = request.form['industry']
        hisi_brand = request.form['brand']
        hisi_description = request.form['description']
        hisi_user = request.form['user']
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('INSERT INTO history_image(product, region, image_type, language, date, industry, brand, user, description) VALUES(?,?,?,?,?,?,?,?,?)',(hisi_product, hisi_region, hisi_image_type, hisi_language, hisi_date, hisi_industry, hisi_brand, hisi_user, hisi_description))
        conn.commit()
        conn.close()
        return render_template('image.html', genPro=hisi_product, genReg=hisi_region, genImg=hisi_image_type, genLan=hisi_language, genDat=hisi_date, genInd=hisi_industry, genBra=hisi_brand, genUse=hisi_user, genDes=hisi_description)
    else:

        return render_template('image.html')


@app.route('/source', methods=['GET','POST'])
def source():
    if request.method == 'POST':
        hiss_source = request.form['source']
        hiss_provider = request.form['provider']
        hiss_division = request.form['division']
        hiss_region = request.form['region']
        hiss_industry = request.form['industry']
        hiss_user = request.form['user']
        hiss_date = request.form['date'] 
        hiss_description = request.form['description']
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('INSERT INTO history_source(source, provider, division, region, industry, user, date, description) VALUES(?,?,?,?,?,?,?,?)',(hiss_source, hiss_provider, hiss_division, hiss_region, hiss_industry, hiss_user, hiss_date, hiss_description))
        conn.commit()
        conn.close()
        return render_template('source.html', genSou=hiss_source, genPro=hiss_provider, genDiv=hiss_division, genReg=hiss_region, genInd=hiss_industry, genUse=hiss_user, genDat=hiss_date, genDes=hiss_description)
    else:

        return render_template('source.html')



@app.route('/campaign', methods=['GET', 'POST'])
def campaign():
    if request.method == 'POST':
        hisc_date = request.form['date']
        hisc_division = request.form['division']
        hisc_camtype = request.form['camtype']
        hisc_industry = request.form['industry']
        hisc_region = request.form['region']
        hisc_product = request.form['product']
        hisc_user = request.form['user']
        hisc_description = request.form['description']
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('INSERT INTO history_campaign(date, division, camtype, industry, region, product, user, description) VALUES(?,?,?,?,?,?,?,?)',(hisc_date, hisc_division, hisc_camtype, hisc_industry, hisc_region, hisc_product, hisc_user, hisc_description))
        conn.commit()
        conn.close()
        return render_template('campaign.html', genPro=hisc_product, genReg=hisc_region, genCam=hisc_camtype, genDiv=hisc_division, genDat=hisc_date, genInd=hisc_industry, genUse=hisc_user, genDes=hisc_description)
    else:

        return render_template('campaign.html')



# Defining Registration Form Class
class RegisterForm(Form):
    fullname = StringField('fullname', [validators.length(min=5)])
    email = StringField('email', [validators.length(min=5)])
    phone = StringField('phone', [validators.length(min=5)])
    zip_code = StringField('zip_code', [validators.length(min=2)])
    street = StringField('street', [validators.length(min=5)])
    city = StringField('city', [validators.length(min=2)])
    country = StringField('country', [validators.length(min=2)])
    username = StringField('username', [validators.length(min=4, max=25)])
    password = PasswordField('password', [
        validators.DataRequired(),
        validators.EqualTo('confirm', message='Ups... Passwords do not match')
    ])
    confirm = PasswordField('confirm')


# Defining Registration route function
@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegisterForm(request.form)
    if request.method == 'POST' and form.validate():
        fullname = form.fullname.data
        email = form.email.data
        phone = form.phone.data
        zip_code = form.zip_code.data
        street = form.street.data
        city = form.city.data
        country = form.country.data
        username = form.username.data
        password = form.password.data
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('INSERT INTO admin(fullname,username,password) VALUES(?,?,?)', (fullname,username,password))
        c.execute('INSERT INTO admin_details(email,phone,zip_code) VALUES(?,?,?)', (email,phone,zip_code))
        c.execute('INSERT INTO zip_code(zip_code,street,city,country) VALUES(?,?,?,?)', (zip_code,street,city,country))
        conn.commit()
        conn.close()

        return render_template('register.html', form=form, success="New admin is registered!!")
    return render_template('register.html', form=form)


# Defining New User Form Class
class NewUser(Form):
    user = StringField('user', [validators.length(min=2)])
    department = StringField('department', [validators.length(min=5)])


# Defining new user route function
@app.route('/newuser', methods=['GET', 'POST'])
def newuser():
    form = NewUser(request.form)
    if request.method == 'POST' and form.validate():
        user = form.user.data
        department = form.department.data
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('INSERT INTO user(user,department) VALUES(?,?)', (user,department))
        conn.commit()
        conn.close()

        return render_template('newuser.html', form=form, success="New user is added!!")
    return render_template('newuser.html', form=form)


# Defining Login route function
@app.route('/login', methods = ['GET','POST'])
def login():
    if request.method == 'POST':
        user_name = request.form ['username']
        password_candidate = request.form['password']
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('SELECT * FROM admin WHERE username = ?', [user_name])
        data = c.fetchone()
        password = data[3]

        #Compare passwords
        if (password_candidate == password):
            print('Password matched')
        else:
            print('Password not matched')
            return render_template('login.html')

        return render_template('admin.html')
    return render_template('login.html')



if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app) 

ModuleNotFoundError: No module named 'flask_wtf'